In [1]:
#DOCUMENTATION - https://docs.trackstarhq.com/introduction
import requests
import pandas as pd
import os
from pathlib import Path

TRACKSTAR_API_KEY = "0ed958207b504d14a0b501bbb73519c5"

# Files will be saved in the current directory
print(f"CSV files will be saved in: {Path.cwd()}")

CSV files will be saved in: /Users/lj/Documents/Code/Deep Ocean Partners Projects/wms-analysis


In [2]:
#Get this information from the web portal 
customer_id = "shipcalm_lookoptic"
connection_id = "ab896808e6d1495b82dbfdf9f8435572"
integration_name = "Infoplus"
access_token = "trackstar_eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJjb25uZWN0aW9uX2lkIjoiYWI4OTY4MDhlNmQxNDk1YjgyZGJmZGY5Zjg0MzU1NzIiLCJpbnRlZ3JhdGlvbl9uYW1lIjoiaW5mb3BsdXMifQ.75BBuw4TwKaccpXN_xWrOCyO1PkkwlTGgBk1LLv7UIA"

In [3]:
#Get available actions for the integration
url = "https://production.trackstarhq.com/wms/integrations/"+integration_name.lower()
headers = {"x-trackstar-api-key": TRACKSTAR_API_KEY}
response = requests.request("GET", url, headers=headers)
#response.json()['data'][0]['available_actions']

In [4]:
def paginate_endpoint(endpoint,page_token):
    print(page_token)
    url = "https://production.trackstarhq.com/wms/"+endpoint
    headers = {
            "x-trackstar-api-key": TRACKSTAR_API_KEY,
            "x-trackstar-access-token": access_token
        }

    if page_token:
        querystring = {"page_token":page_token}
        response = requests.request("GET", url, headers=headers,params=querystring)
    else:
        response = requests.request("GET", url, headers=headers)
    
    return response

In [5]:
def get_data(endpoint):
    output = []
    response = paginate_endpoint(endpoint,"")
    output = output + response.json()['data']
    page_token = response.json()['next_token']
    while page_token:
        response = paginate_endpoint(endpoint,page_token)
        output = output + response.json()['data']
        page_token = response.json()['next_token']
    return output

In [6]:
###########################
# Inventory
############################
output = get_data("inventory")
for row in output:
    row["substitute_skus"]           = str(row["substitute_skus"])
    row["inventory_by_warehouse_id"] = str(row["inventory_by_warehouse_id"])
    row["lots"]                      = str(row["lots"])
    row["measurements"]              = str(row["measurements"])
    row["locations"]                 = str(row["locations"])
df = pd.DataFrame.from_records(output,index=["id","warehouse_customer_id","created_date","updated_date","name","sku","active","awaiting","onhand","committed","unfulfillable","fulfillable","unsellable","sellable","substitute_skus","inventory_by_warehouse_id","lots","measurements","locations","external_system_url"]) 
df.to_csv("inventory.csv")   


eyJpZCI6ICJMQTQwQ0FSMTVfMjA0OTQiLCAiY29ubmVjdGlvbl9pZCI6ICJhYjg5NjgwOGU2ZDE0OTViODJkYmZkZjlmODQzNTU3MiJ9


In [7]:
###########################
# Products
############################
output = get_data("products")
for row in output:
    row["inventory_items"]   = str(row["inventory_items"])
    row["supplier_products"] = str(row["supplier_products"])
df = pd.DataFrame.from_records(output,index=["id","warehouse_customer_id","created_date","updated_date","name","sku","gtin","inventory_items","is_kit","active","supplier","country_of_origin","harmonized_code","supplier_products","external_system_url"]) 
df.to_csv("products.csv")


eyJpZCI6ICJMQTQwQlRQRTMwXzIwNDk0IiwgImNvbm5lY3Rpb25faWQiOiAiYWI4OTY4MDhlNmQxNDk1YjgyZGJmZGY5Zjg0MzU1NzIifQ==


In [8]:
############################
# Shipping Methods
############################
output = get_data("shipping-methods")
df = pd.DataFrame.from_records(output,index=['id', 'created_date','updated_date','name','carrier']) 
df.to_csv("shipping_methods.csv")

In [9]:
############################
# Shipment Suppliers
############################
output = get_data("inbound-shipments/suppliers")
df = pd.DataFrame.from_records(output,index=['id', 'created_date','updated_date']) 
df.to_csv("suppliers.csv")

In [10]:
############################
# Inbound Shipments
############################
output = get_data("inbound-shipments")
for row in output:
    row["line_items"]         = str(row["line_items"])
    row["receipts"]           = str(row["receipts"])
    row["ship_from_address"]  = str(row["ship_from_address"])
    row["tracking_numbers"]   = str(row["tracking_numbers"])
    row["notes"]              = str(row["notes"])
df = pd.DataFrame.from_records(output,index=["id","warehouse_customer_id","created_date","updated_date","purchase_order_number","status","raw_status","supplier","expected_arrival_date","warehouse_id","line_items","receipts","ship_from_address","tracking_numbers","notes","external_system_url"]) 
df.to_csv("inbound_shipments.csv")

In [11]:
############################
# Order Channels
############################
output = get_data("orders/channels")
#for row in output:
#    row["line_items"]         = str(row["line_items"])
#    row["receipts"]           = str(row["receipts"])
#    row["tracking_numbers"]   = str(row["tracking_numbers"])
#    row["notes"]              = str(row["notes"])
#df = pd.DataFrame.from_records(output,index=["id","warehouse_customer_id","created_date","updated_date","purchase_order_number","status","raw_status","supplier","expected_arrival_date","warehouse_id","line_items","receipts","ship_from_address","tracking_numbers","notes","external_system_url"]) 
#df.to_csv("/Users/danieljacobini/Downloads/inbound_shipments.csv")

In [12]:
############################
# Orders
############################
output = get_data("orders")
print(len(output))


...
356685


In [13]:
line_items = []
for row in output:
    line_items = line_items + [dict(item, **{'id':row["id"]}) for item in row["line_items"]]
df = pd.DataFrame.from_records(line_items,index=["product_id","sku","quantity","unit_price","is_picked","discount_amount","id"])
df.to_csv("orders_line_items.csv")

In [14]:
for row in output:
    row["ship_to_address"] = str(row["ship_to_address"])
    row["line_items"]      = str(row["line_items"])
    row["tags"]            = str(row["tags"])
    row["shipments"]       = str(row["shipments"])
df = pd.DataFrame.from_records(output,index=["id","warehouse_customer_id","created_date","updated_date","reference_id","order_number","status","raw_status","channel","type","trading_partner","shipping_method","invoice_currency_code","total_price","total_tax","total_discount","total_shipping","ship_to_address","line_items","tags","required_ship_date","shipments","external_system_url"]) 
df.to_csv("orders.csv")